In [7]:
import math
import pandas as pd

# Load the CSV file
file_path = "W_Section_Name.csv"
w_sections_df = pd.read_csv(file_path)

# Convert the DataFrame to a dictionary for structural analysis
w_sections_data = w_sections_df.set_index("Section").T.to_dict()

def analyze_and_select_economical_section(Fx, Fy, Fz, Mx, My, Mz, L, Lb, Fy_steel=345, E=200000):
    """
    Analyze and select the most economical W-section based on capacity-to-demand ratio.
    """
    phi = 0.9  # Resistance factor for flexure
    phi_v = 0.75  # Resistance factor for shear
    results = []

    for section_name, properties in w_sections_data.items():
        d = properties["d"]  # mm
        tw = properties["tw"]  # mm
        Ix = properties["Ix"] * 1e6  # mm^4
        Zx = properties["Zx"] * 1e3  # mm³
        Sx = properties["Sx"] * 1e3  # mm³

        # Shear strength
        Aw = d * tw  # mm²
        Vn = 0.6 * Fy_steel * Aw / 1e3  # kN

        # Flexural capacity
        M_yield = Fy_steel * Zx / 1e6  # kN-m
        Lb_mm = Lb * 1000  # mm
        Lp = 1.76 * math.sqrt(E / Fy_steel) * (d / 2)  # mm (approximation for Lp)
        Lr = 1.95 * math.sqrt(E / Fy_steel) * (d / 2)  # mm (approximation for Lr)

        if Lb_mm <= Lp:
            M_ltb = M_yield  # No LTB
        elif Lp < Lb_mm <= Lr:
            M_ltb = M_yield * (1 - 0.3 * (Lb_mm - Lp) / (Lr - Lp))
        else:
            M_ltb = 1.17 * math.pi**2 * E * Sx / (Lb_mm**2 * 1e6)

        M_capacity = min(M_yield, M_ltb)  # kN-m
        V_capacity = phi_v * Vn  # kN

        # Serviceability (Deflection)
        delta_max = (L * 1000) / 360  # mm
        delta_actual = (5 * Fz * (L * 1e3)**3) / (384 * E * Ix)  # mm

        if M_capacity < Mx or V_capacity < Fz or delta_actual > delta_max:
            continue  # Skip if any constraint is violated

        # Append results
        M_utilization = Mx / M_capacity if M_capacity > 0 else float("inf")
        V_utilization = Fz / V_capacity if V_capacity > 0 else float("inf")
        results.append({
            "Section": section_name,
            "Weight (kg/m)": properties["Zx"] / 1000,  # Proxy for weight
            "M_capacity (kN-m)": round(M_capacity, 2),
            "V_capacity (kN)": round(V_capacity, 2),
            "M_utilization": round(M_utilization, 2),
            "V_utilization": round(V_utilization, 2),
            "Delta_actual (mm)": round(delta_actual, 2),
            "Delta_max (mm)": round(delta_max, 2),
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    if not results_df.empty:
        # Sort by M_utilization closest to 1.0
        results_df["Optimization"] = (results_df["M_utilization"] - 1.0).abs()
        results_df = results_df.sort_values(by="Optimization", ascending=True)
        return results_df.iloc[0]
    return None


# Input parameters
Fx = 0    # Axial load (kN)
Fy = 0    # Shear load (kN)
Fz = 111   # Vertical load (kN)
Mx = 333   # Moment about x-axis (kN-m)
My = 0    # Moment about y-axis (kN-m)
Mz = 0    # Moment about z-axis (kN-m)
L = 6     # Span length (m)
Lb = 6    # Unbraced length (m)

# Find the most economical section
economical_section = analyze_and_select_economical_section(Fx, Fy, Fz, Mx, My, Mz, L, Lb)

# Display the most economical section
if economical_section is not None:
    print(f"Most economical section: {economical_section['Section']}")
    print(economical_section)
else:
    print("No suitable section found.")


Most economical section: W14X38
Section              W14X38
Weight (kg/m)          1.01
M_capacity (kN-m)    348.45
V_capacity (kN)      437.41
M_utilization          0.96
V_utilization          0.25
Delta_actual (mm)      0.01
Delta_max (mm)        16.67
Optimization           0.04
Name: 206, dtype: object
